In [23]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
import time

In [24]:
iris = pd.read_csv('iris (1).data')
X_iris = iris.iloc[:, :-1].values
y_iris = iris.iloc[:, -1].values

In [ ]:
mnist = fetch_openml('mnist_784', version=1)
X_mnist, _, y_mnist, _ = train_test_split(mnist.data, mnist.target, train_size=2000, stratify=mnist.target, random_state=42)

In [14]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)
X_train_mnist, X_test_mnist, y_train_mnist, y_test_mnist = train_test_split(X_mnist, y_mnist, test_size=0.3, stratify=y_mnist, random_state=42)

In [25]:
print("Shapes of Iris dataset : ", X_train_iris.shape, X_test_iris.shape)
print("Shapes of MNIST subset : ", X_train_mnist.shape, X_test_mnist.shape)

Shapes of Iris dataset :  (104, 4) (45, 4)
Shapes of MNIST subset :  (1400, 784) (600, 784)


In [17]:
def evaluate_dimensionality_reduction(X_train, X_test, y_train, y_test, method, tree_params, **kwargs):
    results = []
    if method == 'PCA':
        model = PCA(**kwargs)
    elif method == 'LDA':
        model = LDA(**kwargs)
    elif method == 'KernelPCA':
        model = KernelPCA(**kwargs)
    else:
        raise ValueError("Invalid method")

    X_train_reduced = model.fit_transform(X_train, y_train)
    X_test_reduced = model.transform(X_test)

    for params in tree_params:
        classifier = DecisionTreeClassifier(**params, random_state=42)
        start_time = time.time()
        classifier.fit(X_train_reduced, y_train)
        train_time = time.time() - start_time
        y_pred = classifier.predict(X_test_reduced)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        results.append({
            'method': method,
            'tree_params': params,
            'train_time': train_time,
            'accuracy': accuracy,
            'f1_score': f1,
            'n_components': X_train_reduced.shape[1]
        })
    return results

In [18]:
params = {
    'PCA': {'n_components': 2},
    'LDA': {'n_components': 2},
    'KernelPCA': {'n_components': 2, 'kernel': 'rbf'}
}
tree_params = [{'max_depth': 3}, {'max_depth': 5}, {'min_samples_split': 4}, {'min_samples_leaf': 2}]

In [27]:
iris_results = []
for method, kwargs in params.items():
    iris_results.extend(evaluate_dimensionality_reduction(X_train_iris, X_test_iris, y_train_iris, y_test_iris, method, tree_params, **kwargs))

In [28]:
mnist_results = []
for method, kwargs in params.items():
    mnist_results.extend(evaluate_dimensionality_reduction(X_train_mnist, X_test_mnist, y_train_mnist, y_test_mnist, method, tree_params, **kwargs))

In [29]:
(iris_results)

[{'method': 'PCA',
  'tree_params': {'max_depth': 3},
  'train_time': 0.0016713142395019531,
  'accuracy': 0.8888888888888888,
  'f1_score': 0.8873896385140763,
  'n_components': 2},
 {'method': 'PCA',
  'tree_params': {'max_depth': 5},
  'train_time': 0.005698442459106445,
  'accuracy': 0.9111111111111111,
  'f1_score': 0.9105820105820105,
  'n_components': 2},
 {'method': 'PCA',
  'tree_params': {'min_samples_split': 4},
  'train_time': 0.0015077590942382812,
  'accuracy': 0.9111111111111111,
  'f1_score': 0.9105820105820105,
  'n_components': 2},
 {'method': 'PCA',
  'tree_params': {'min_samples_leaf': 2},
  'train_time': 0.0015146732330322266,
  'accuracy': 0.9111111111111111,
  'f1_score': 0.9105820105820105,
  'n_components': 2},
 {'method': 'LDA',
  'tree_params': {'max_depth': 3},
  'train_time': 0.005197286605834961,
  'accuracy': 0.9777777777777777,
  'f1_score': 0.9777448559670783,
  'n_components': 2},
 {'method': 'LDA',
  'tree_params': {'max_depth': 5},
  'train_time': 0.

In [30]:
(mnist_results)

[{'method': 'PCA',
  'tree_params': {'max_depth': 3},
  'train_time': 0.029590368270874023,
  'accuracy': 0.37833333333333335,
  'f1_score': 0.2991973651103694,
  'n_components': 2},
 {'method': 'PCA',
  'tree_params': {'max_depth': 5},
  'train_time': 0.012371063232421875,
  'accuracy': 0.44333333333333336,
  'f1_score': 0.41958062653634975,
  'n_components': 2},
 {'method': 'PCA',
  'tree_params': {'min_samples_split': 4},
  'train_time': 0.016677379608154297,
  'accuracy': 0.3883333333333333,
  'f1_score': 0.38646070162107454,
  'n_components': 2},
 {'method': 'PCA',
  'tree_params': {'min_samples_leaf': 2},
  'train_time': 0.016287565231323242,
  'accuracy': 0.38,
  'f1_score': 0.37438004599700675,
  'n_components': 2},
 {'method': 'LDA',
  'tree_params': {'max_depth': 3},
  'train_time': 0.01009225845336914,
  'accuracy': 0.36666666666666664,
  'f1_score': 0.2609154869601054,
  'n_components': 2},
 {'method': 'LDA',
  'tree_params': {'max_depth': 5},
  'train_time': 0.011090278625